In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.6/808.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.0 MB/s eta 0:00:00


In [ ]:
!wget https://huggingface.co/localmodels/Llama-2-7B-Chat-ggml/resolve/main/llama-2-7b-chat.ggmlv3.q4_K_S.bin


--2023-12-12 18:09:50--  https://huggingface.co/localmodels/Llama-2-7B-Chat-ggml/resolve/main/llama-2-7b-chat.ggmlv3.q4_K_S.bin
Resolving huggingface.co (huggingface.co)... 54.230.18.95, 54.230.18.84, 54.230.18.85, ...
Connecting to huggingface.co (huggingface.co)|54.230.18.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/94/0c/940c322080835ee5b3ac017aa3c46f8ee7742b8a285c0be10beb5428d7186ee0/32b758bf5e4f16fb5944b75d577fbca18c11c57000b41c6cc04bb281632d58f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q4_K_S.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q4_K_S.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1702663790&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjY2Mzc5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85NC8wYy85NDBjMzIyMDgwODM1ZWU1YjNhYzAxN2FhM2M0NmY4ZWU3NzQyYjhhMjg1YzBiZTEwYm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install llama-cpp-python==0.1.78
#RESTART AFTER THIS
!pip install faiss-gpu

In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
llm = LlamaCpp(model_path="llama-2-7b-chat.ggmlv3.q4_K_S.bin", n_ctx=2048)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


Invoking llama7b similar to chatgpt

In [ ]:
template = """
              As a healthcare professional, review the patient's initial and subsequent hospital visit notes separated by $$$$ enclosed in triple backquotes.
              Provide a brief explanation for the cause of readmission.

              ```{text1}, $$$$ {text2}```
              Stay concise.
           """
prompt = PromptTemplate(template=template, input_variables=["text1", "text2"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

clinical_note = {"text1": """"dmiNeonatology Attending Triage Note\\n\\nBaby [**Name (NI) 1**] [**Known lastname 2**] is a term male infant atted to the NICU for sepsis evaluation.  Asked to evaluate baby by Dr. [**First Name4 (NamePattern1) 3**] [**Last Name (NamePattern1) 4**].\\n\\nMother is 34 years old G1 P0-1.\\nPNS:  A pos, Ab neg, HBSAg neg, RPR NR, RI, GBS neg.\\nPregnancy was uncomplicated.\\nDelivery was by C-section after failure to progress.  Apgars 9,9.\\nMother was treated with antibiotics because of maternal temp of 100.3 just prior to delivery.  Mother\'s temp was then lower but at 2 hours rose again to 102.\\n\\nPE - Baby is [**Name2 (NI) 5**] and vigorous, AGA.\\nVS -  T 98.5 HR 145 RR 38 BP 72/35 48, O2 sat 100% in RA\\nHEENT - AF soft and flat, some molding noted.\\nPalate intact.\\nResp - breath sounds clear and equal.\\nCVS - S1 S2 normal, [**3-18**] high pitched systolic murmur at LSB\\nAbd - soft with normal bowel sounds, no organomegaly\\nGU - normal male with testes descended bilaterally\\nNeuro - tone wnl, symmetrical exam\\n\\nDS - 72\\n\\nAssessment/plan:\\nTerm male infant with increased risk of sepsis.\\nWill check cbc diff and plats, blood culture.\\nWill cover with antibiotics for at least 48 hours pending results of cultures.\\nFurther work up with possible LP if culture is positive or clinical signs of sepsis develop.\\n\\n\\n\\n""", "text2": """Nursing Transfer note\\n\\n\\nPt admitted to NICU for sepsis eval. Please see Attending\\nnote for details regarding maternal history and delivery\\ndetails.\\n\\nInfant stable in RA. RR 30-40\'s, sats 96-100%. LS clear/=.\\nNo retractions noted. HR 140\'s. No murmur. Infant [** 5**], well\\nperfused. BW 3865g. CBC and BC sent, pending at this time.\\nInfant on 48 r/o sepsis with abx Amp and Gent. PIV placed in\\nLeft hand, meds administered as ordered. D Stick 72. Infant\\nstable for transfer to NBN. Continue to monitor for s/s of\\nsepsis.\\n\\n\\n"""}
answer = llm_chain.run(clinical_note)
print(answer)

 Please explain the cause of readmission, based on the information provided in the hospital visit notes enclosed by triple backquotes.$$$$ 


RAG example

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.embeddings import LlamaCppEmbeddings


In [ ]:
import pandas as pd
vector_store_db = pd.read_csv('/content/drive/My Drive/vector_store_rag.csv')

In [ ]:
vector_store_db

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
...,...,...,...
16402,symptoms,What are the symptoms of Familial visceral myo...,What are the signs and symptoms of Familial vi...
16403,information,What is (are) Pseudopelade of Brocq ?,Pseudopelade of Brocq (PBB) is a slowly progre...
16404,symptoms,What are the symptoms of Pseudopelade of Brocq ?,What are the signs and symptoms of Pseudopelad...
16405,treatment,What are the treatments for Pseudopelade of Br...,Is there treatment or a cure for pseudopelade ...


In [ ]:
data_list = vector_store_db.to_dict(orient='records')

In [ ]:
data_list = [str(ele) for ele in data_list]

In [ ]:
final_string = "******".join(data_list)

In [ ]:
# How about a physical exam with some vitals and impressions?
llama = LlamaCppEmbeddings(model_path="llama-2-7b-chat.ggmlv3.q4_K_S.bin")

cause_report_pairs = []

vectorstore = FAISS.from_texts(final_string,  embedding=llama)
retriever = vectorstore.as_retriever()

template = """You are given two medical reports summary.
              First one is from the previous encounter discharge summary and the second one is the note from the next readmission.
              Report1: {text1}
              Report2: {text2}
              Answer What is the cause for readmission for the given patient and using the {context}- stay pretty succint.
           """
prompt = ChatPromptTemplate.from_template(template)
model = llm

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
vectorstore

In [ ]:
chain = ( {"context": retriever, "text1": RunnablePassthrough(), "text2": RunnablePassthrough()} | prompt  | model | StrOutputParser() )

text1 = """An 87-year-old woman with pelvic pain was seen by a radiologist. The patient had multiple chronic medical issues. She had a large hiatal hernia, a small pleural effusion, and a large stomach tube."""
text2 = """The patient is an 87-year-old Russian speaking female. She was admitted to the hospital on [**2185-2-20**] to the Vascular Service with a nonhealing ulcer on her fourth right toe x3 months. She underwent a fem-[**Doctor Last Name **] bypass by Vascular Surgery. Postoperatively in the PACU, the patient developed some respiratory distress and had a PEA arrest with very difficult intubation."""

clinical_note = {"text1": text1, "text2": text2}

chain.invoke((text1, text2))

' }\n```\nBased on the provided medical reports, the cause of readmission for the patient is:\n\n* Respiratory distress after fem-[Doctor Last Name] bypass surgery in the postoperative period.\n\nThe patient was readmitted to the hospital due to respiratory distress developed after the surgery, which suggests that there may have been some complications during or after the procedure.'

In [ ]:
#testing with HF models
!pip install transformers bitsandbytes>=0.39.0 -q

In [ ]:
!pip install accelerate -U
!pip -qqq install bitsandbytes accelerate
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]